In [ ]:
import os
import glob
import json
import pandas as pd
import numpy as np
from collections import defaultdict
import collections as coll
import re

In [2]:
files = glob.glob("/Users/kkim14/tcr_structure_features/data/emory/raw/*_all_contig_annotations.csv.gz")
files

['/Users/kkim14/tcr_structure_features/data/emory/raw/GSM5456934_HPV7_QVD_metLN_all_contig_annotations.csv.gz',
 '/Users/kkim14/tcr_structure_features/data/emory/raw/GSM5456916_HPV34_KSA_metLN_all_contig_annotations.csv.gz',
 '/Users/kkim14/tcr_structure_features/data/emory/raw/GSM5456908_HPV15_QVD_Tumor_all_contig_annotations.csv.gz',
 '/Users/kkim14/tcr_structure_features/data/emory/raw/GSM5456912_HPV34_QVD_Tumor_all_contig_annotations.csv.gz',
 '/Users/kkim14/tcr_structure_features/data/emory/raw/GSM5456914_HPV34_QVD_metLN_all_contig_annotations.csv.gz',
 '/Users/kkim14/tcr_structure_features/data/emory/raw/GSM5456910_HPV15_KSA_Tumor_all_contig_annotations.csv.gz',
 '/Users/kkim14/tcr_structure_features/data/emory/raw/GSM5456922_HPV51_QVD_Tumor_all_contig_annotations.csv.gz',
 '/Users/kkim14/tcr_structure_features/data/emory/raw/GSM5456943_HPV42_QVD_Tumor_all_contig_annotations.csv.gz',
 '/Users/kkim14/tcr_structure_features/data/emory/raw/GSM5456945_HPV37_QVD_metLN_all_contig_annot

In [48]:
dfs = []

for file in files:
    sample = file.split("/")[-1].replace("_all_contig_annotations.csv.gz", "")
    print(sample)
    df=pd.read_csv(f"{file}", compression='gzip')
    
    df = df[(df['high_confidence']==True) & (df['full_length']==True) & (df['productive']==True) & (df['chain'].isin(['TRA','TRB']))]
    df.dropna(subset=['raw_clonotype_id','v_gene','j_gene'], inplace=True)
    
    df = df.groupby(['raw_clonotype_id','chain','v_gene','j_gene','c_gene','d_gene','cdr3','cdr3_nt'], dropna=False).agg({
        'contig_id':list,
        'barcode':list,
        'umis':list
    }).reset_index()
    
    df_a = df[df['chain']=='TRA']
    df_b = df[df['chain']=='TRB']
    df = df_a.merge(df_b, on='raw_clonotype_id', suffixes=("_a","_b"))

    df.drop(columns=['chain_a','chain_b'], inplace=True)
    df['clonotype_count'] = df.groupby('raw_clonotype_id').cumcount() + 1
    clonotype = df['raw_clonotype_id'].astype(str) + "-" + df['clonotype_count'].astype(str)
    df['clonotype'] = clonotype
    donor_clonotype = sample + "_" + df['clonotype'].astype(str)
    df.insert(0, "donor_clonotype", donor_clonotype)

    df.to_csv(f"/Users/kkim14/tcr_structure_features/data/emory/raw/{sample}_consensus_clonotypes.csv", index=False)
    dfs.append(df)

GSM5456934_HPV7_QVD_metLN
GSM5456916_HPV34_KSA_metLN
GSM5456908_HPV15_QVD_Tumor
GSM5456912_HPV34_QVD_Tumor
GSM5456914_HPV34_QVD_metLN
GSM5456910_HPV15_KSA_Tumor
GSM5456922_HPV51_QVD_Tumor
GSM5456943_HPV42_QVD_Tumor
GSM5456945_HPV37_QVD_metLN
GSM5456936_HPV7_KSA_metLN
GSM5456932_HPV7_KSA_Tumor
GSM5456924_HPV51_KSA_Tumor
GSM5456926_HPV51_QVD_metLN


In [49]:
dfs = pd.concat(dfs)
dfs['v_gene_a'] = dfs['v_gene_a'].astype(str).str.replace("DV","/DV")
dfs['v_gene_b'] = dfs['v_gene_b'].astype(str).str.replace("DV","/DV")
dfs.shape

(1807, 22)

In [7]:
import importlib.resources as importlib_resources

data_files = importlib_resources.files("Data")
additional_genes_file = str(data_files / 'additional-genes.fasta')
linkers_file = str(data_files / 'linkers.tsv')
data_dir = os.path.dirname(additional_genes_file)
gui_examples_dir = os.path.join(data_dir, 'GUI-Examples')


regions = {'l': 'LEADER',
           'v': 'VARIABLE',
           'j': 'JOINING',
           'c': 'CONSTANT'
           }

def nest():
    """
    Create nested defaultdicts
    """
    return coll.defaultdict(list)

def read_fa(ff):
    """
    :param ff: opened fasta file
    read_fa(file):Heng Li's Python implementation of his readfq function (tweaked to only bother with fasta)
    https://github.com/lh3/readfq/blob/master/readfq.py
    """

    last = None                                 # this is a buffer keeping the last unprocessed line
    while True:                                 # mimic closure
        if not last:                            # the first record or a record following a fastq
            for l in ff:                        # search for the start of the next record
                if l[0] in '>':                 # fasta header line
                    last = l[:-1]               # save this line
                    break
        if not last:
            break
        name, seqs, last = last[1:], [], None
        for l in ff:                            # read the sequence
            if l[0] in '>':
                last = l[:-1]
                break
            seqs.append(l[:-1])
        if not last or last[0] != '+':          # this is a fasta record
            yield name, ''.join(seqs), None     # yield a fasta record
            if not last:
                break
        else:
            raise IOError("Input file does not appear to be a FASTA file - please check and try again")



def get_ref_data(tcr_chain, gene_types, species):
    """
    :param tcr_chain: 3 digit str code, e.g. TRA or TRB
    :param gene_types: list of TYPES of genes to be expected in a final TCR mRNA, in their IMGT nomenclature
    :param species: upper case str, for use if a specific absolute path not specified
    :return: triply nested dict of TCR data: { region { gene { allele { seq } } }; a doubly nested dict with V/J
      functionalities, and a doubly nested dict of genes filtered out due to being partial in their 5' or 3' (or both)
    """

    # Run some basic sanity/input file checks
    if tcr_chain not in ['TRA', 'TRB', 'TRG', 'TRD', 'IGH', 'IGL', 'IGK']:
        raise ValueError("Incorrect chain detected (" + tcr_chain + "), cannot get reference data. ")

    in_file_path = os.path.join(data_dir, species, tcr_chain + '.fasta')
    if not os.path.isfile(in_file_path):
        raise IOError(tcr_chain + '.fasta not detected in the Data directory. '
                                  'Please check data exists for this species/locus combination. ')

    # Read in the data to a nested dict
    tcr_data = {}
    for gene_type in gene_types:
        tcr_data[gene_type] = coll.defaultdict(nest)

    functionality = coll.defaultdict(nest)
    partial_genes = coll.defaultdict(nest)

    with open(in_file_path, 'r') as in_file:
        for fasta_id, seq, blank in read_fa(in_file):
            bits = fasta_id.split('|')
            if len(bits) < 13:
                raise IOError("Input TCR FASTA file does not fit the IMGT header format. ")

            gene, allele = bits[1].split('*')
            functionality_call = bits[3]
            seq_type = fasta_id.split('~')[1]
            partial_flag = bits[13]

            functionality[gene][allele] = functionality_call

            if 'partial' in partial_flag:
                partial_genes[gene][allele] = partial_flag
            else:
                tcr_data[seq_type][gene][allele] = seq.upper()

    for gene_type in gene_types:
        if len(tcr_data[gene_type]) == 0:
            raise Exception("No entries for " + gene_type + " in IMGT data. ")

    return tcr_data, functionality, partial_genes



def get_additional_genes(imgt_data, imgt_functionality):
    """
    :param imgt_data: the nested dict produced by get_ref_data containing V/J/C sequence data
    :param imgt_functionality: the nested dict with imgt_stated functionality
    :return: the same dicts supplemented with any genes found in the 'additional genes.fasta' file
    """

    with open(additional_genes_file, 'r') as in_file:
        for fasta_id, seq, blank in read_fa(in_file):
            bits = fasta_id.split('|')

            if len(bits) < 5:
                raise IOError("Sequence in additional-genes.fasta doesn't have enough fields in header: " + fasta_id)

            if '*' in bits[1]:
                gene, allele = bits[1].upper().split('*')
            else:
                raise IOError("Sequence in additional-genes.fasta doesn't have correct gene name format ('" + bits[1]
                              + "'): " + fasta_id)

            if bits[3]:
                functionality_call = bits[3].replace('(', '').replace(')', '').replace('[', '').replace(']', '')
            else:
                functionality_call = 'F'

            if '~' in fasta_id:
                seq_type = fasta_id.split('~')[1]
                if seq_type not in regions.values():
                    raise IOError("Sequence in additional-genes.fasta doesn't have valid gene type ('" + seq_type + "')"
                                  ": " + fasta_id)
            else:
                raise IOError("Sequence in additional-genes.fasta doesn't have the required '~' character: " + fasta_id)

            imgt_data[seq_type][gene][allele] = seq
            imgt_functionality[gene][allele] = functionality_call

    return imgt_data, imgt_functionality


gene_types = list(regions.values())
a_imgt_dat, a_tcr_functionality, a_partial = get_ref_data('TRA', gene_types, 'HUMAN')
a_imgt_dat, a_tcr_functionality = get_additional_genes(a_imgt_dat, a_tcr_functionality)

b_imgt_dat, b_tcr_functionality, b_partial = get_ref_data('TRB', gene_types, 'HUMAN')
b_imgt_dat, b_tcr_functionality = get_additional_genes(b_imgt_dat, b_tcr_functionality)

In [ ]:

dfs

,TRAV,TRAJ,TRA_CDR3,TRBV,TRBJ,TRB_CDR3,TRAC,TRBC,TRA_leader,TRB_leader,Linker,Link_order,TRA_5_prime_seq,TRA_3_prime_seq,TRB_5_prime_seq,TRB_3_prime_seq
0,TRAV21,TRAJ9,TGTGCTGCCGATACTGGAGGCTTCAAAACTATCTTT,TRBV27,TRBJ1-6,TGTGCCAGCACAGGGGGCGACTATAATTCACCCCTCCACTTT,TRAC,TRBC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRAV21,TRAJ9,TGTGCTGCAGATACTGGAGGCTTCAAAACTATCTTT,TRBV27,TRBJ1-2,TGTGCCAGCAGTCCAGCGGACAGGGGCGATGGCTACACCTTC,TRAC,TRBC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRAV17,TRAJ26,TGTGCTACGGTCGCGATCCGGGATAACTATGGTCAGAATTTTGTCTTT,TRBV19,TRBJ2-1,TGTGCCAGTAGGATGACTAGCGGGGGGGCTCGGAATGAGCAGTTCTTC,TRAC,TRBC2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRAV21,TRAJ9,TGTGCTGTTAATACTGGAGGCTTCAAAACTATCTTT,TRBV19,TRBJ2-1,TGTGCCAGTAGGATGACTAGCGGGGGGGCTCGGAATGAGCAGTTCTTC,TRAC,TRBC2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TRAV21,TRAJ9,TGTGCTGTGGATACTGGAGGCTTCAAAACTATCTTT,TRBV27,TRBJ1-2,TGTGCCAGCAGTTTGGGGGGTACTCTTGACTATGGCTACACCTTC,TRAC,TRBC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,TRAV12-2,TRAJ32,TGTGCCGTGGACAGTGGTGCTACAAACAAGCTCATCTTT,TRBV29-1,TRBJ2-3,TGCAGCGTTGACGGAGGGGTCGGCGTAACTAGCACAGATACGCAGT...,TRAC,TRBC2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,TRAV21,TRAJ9,TGTGCTGTTGATACTGGAGGCTTCAAAACTATCTTT,TRBV27,TRBJ1-2,TGTGCCAGCAGTCCCGCCGACAGGGCGAATGGCTACACCTTC,TRAC,TRBC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,TRAV21,TRAJ9,TGTGCTGTGGATACTGGAGGCTTCAAAACTATCTTT,TRBV27,TRBJ1-2,TGTGCCAGCAGTTCCGGCGACAGGGGGCAGGATGGCTACACCTTC,TRAC,TRBC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,TRAV21,TRAJ9,TGTGCTGCGGATACTGGAGGCTTCAAAACTATCTTT,TRBV6-5,TRBJ1-2,TGTGCCAGCAGTTACTCTGACGGGAACTATGGCTACACCTTC,TRAC,TRBC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
dfs.rename(columns={'v_gene_a': 'TRAV', 'j_gene_a': 'TRAJ', 'cdr3_nt_a': 'TRA_CDR3', 'v_gene_b': 'TRBV', 'j_gene_b': 'TRBJ', 'cdr3_nt_b': 'TRB_CDR3', 'c_gene_a': 'TRAC', 'c_gene_b': 'TRBC'}, inplace=True)
df = dfs.groupby(['TRAV', 'TRAJ', 'TRA_CDR3', 'TRBV', 'TRBJ', 'TRB_CDR3', 'TRAC', 'TRBC'])['donor_clonotype'].apply(list).reset_index()
df.reset_index(names=['TCR_name'], inplace=True)
df['TCR_name'] = "emory_" + df['TCR_name'].astype(str)
df

column_need = ['TCR_name', 'TRAV', 'TRAJ', 'TRA_CDR3', 'TRBV', 'TRBJ', 'TRB_CDR3', 'TRAC', 'TRBC', 
               'TRA_leader', 'TRB_leader', 'Linker', 'Link_order', 'TRA_5_prime_seq', 'TRA_3_prime_seq', 'TRB_5_prime_seq', 'TRB_3_prime_seq']
df = df[['TCR_name', 'TRAV', 'TRAJ', 'TRA_CDR3', 'TRBV', 'TRBJ', 'TRB_CDR3', 'TRAC', 'TRBC']]
df = df.reindex(columns=column_need)
df.to_csv("/Users/kkim14/tcr_structure_features/data/emory/all_tcr_seqs.tsv", sep='\t', index=False)

In [56]:
df = pd.read_csv("/Users/kkim14/tcr_structure_features/data/emory/all_tcr_seqs_stitched.tsv", sep='\t')
print(df['TRA_aa'].isna().sum())
print(df['TRB_aa'].isna().sum())

4
3


In [58]:
df.dropna(subset=['TRA_aa','TRB_aa'], inplace=True)
df[['TCR_name','TRA_aa','TRB_aa']].rename(columns={'TCR_name':'tcr_id','TRA_aa':'tra_aa','TRB_aa':'trb_aa'}).to_csv("/Users/kkim14/tcr_structure_features/data/emory/all_tcr_seqs.csv", index=False)